In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import pickle
from tqdm import tqdm
import gait
from train import CVAE_FULL

/home/hexplex0xff/GAN/gan/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hexplex0xff/GAN/gan/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hexplex0xff/GAN/gan/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hexplex0xff/GAN/gan/lib/python3.6/site-packages/tensorflow/python/f

In [5]:
image_vec_shape = 12
gait_key_poses = 16
conditional_vec_shape = 4
pose_image_shape = (160,160,1)
batch_size = 50
full_vec_shape = image_vec_shape+conditional_vec_shape
n_timesteps = 3
angle = '090'

In [7]:
encoder = keras.models.Model(CVAE_FULL.input, CVAE_FULL.get_layer('concat_zcond').output)
encoded_vec = gait.encode_data(encoder,label_angle='090')

100%|██████████| 124/124 [00:06<00:00, 18.42it/s]


In [8]:
class GANDataGenerator(keras.utils.Sequence):
    def __init__(self, generator, encoded_vec, preprocess=None,  batch_size = 50, full_vec_shape=(1,16), dim=(160,160), n_classes=16, shuffle=True):
        self.dim = dim
        self.encoded_vec = encoded_vec
        self.full_vec = full_vec_shape
        self.batch_size = batch_size
        self.gen = generator
        self.preprocess = preprocess or (lambda x : x)
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def correlate(self):
        for folder in self.encoded_vec:
            i = 0
            for file in sorted(os.listdir(folder)):
                if file[:-3]!='pkl' and file[:-3]!='npy' and file[:-3]!='npz':
                    
                    i += 1
            
        self.dataset = list(zip())
        
    def __len__(self):
        return int(np.floor(len(self.files)/self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        files_temp = [self.files[k] for k in indexes]
        
        X,z = self.__data_generation(files_temp)
        return [X,z], None
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.files))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, files_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        z = np.empty((self.batch_size),dtype=int)
        for i,file in enumerate(files_temp):
            X[i,] = cv2.copyMakeBorder(self.preprocess(cv2.imread(file)), 0, 0, 20, 20, cv2.BORDER_CONSTANT, (0,0,0)).reshape(*self.dim, self.n_channels)/255.
            z[i] = self.labels[file]
        return X, keras.utils.to_categorical(z, num_classes=self.n_classes)

In [12]:
encoded_vec['/home/hexplex0xff/GAN/GaitDatasetB-silh/001/bg-01/090'].shape

(59, 16)

In [16]:
import os

In [18]:
sorted(os.listdir('/home/hexplex0xff/GAN/GaitDatasetB-silh/001/bg-01/090'))

['001-bg-01-090-035.png',
 '001-bg-01-090-036.png',
 '001-bg-01-090-037.png',
 '001-bg-01-090-038.png',
 '001-bg-01-090-039.png',
 '001-bg-01-090-040.png',
 '001-bg-01-090-041.png',
 '001-bg-01-090-042.png',
 '001-bg-01-090-043.png',
 '001-bg-01-090-044.png',
 '001-bg-01-090-045.png',
 '001-bg-01-090-046.png',
 '001-bg-01-090-047.png',
 '001-bg-01-090-048.png',
 '001-bg-01-090-049.png',
 '001-bg-01-090-050.png',
 '001-bg-01-090-051.png',
 '001-bg-01-090-052.png',
 '001-bg-01-090-053.png',
 '001-bg-01-090-054.png',
 '001-bg-01-090-055.png',
 '001-bg-01-090-056.png',
 '001-bg-01-090-057.png',
 '001-bg-01-090-058.png',
 '001-bg-01-090-059.png',
 '001-bg-01-090-060.png',
 '001-bg-01-090-061.png',
 '001-bg-01-090-062.png',
 '001-bg-01-090-063.png',
 '001-bg-01-090-064.png',
 '001-bg-01-090-065.png',
 '001-bg-01-090-066.png',
 '001-bg-01-090-067.png',
 '001-bg-01-090-068.png',
 '001-bg-01-090-069.png',
 '001-bg-01-090-070.png',
 '001-bg-01-090-071.png',
 '001-bg-01-090-072.png',
 '001-bg-01-